<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/copy_zip_to_GCS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This script will point to my drive and extract the zip files from the wetland_unet folder and copy them here, to local disk. Then it will connect to GCS bucket and copy the unziped files. After that, I moved the files from subfolders to a top level "subset_data" folder in the bucket.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.3 MB/s eta 0:00:00


In [4]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode

# Additional code can be added here if needed

In [5]:
directory_path = '/content/image_subsets'
os.makedirs(directory_path, exist_ok=True)

In [6]:
zip_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.zip')
extract_dir = '/content/image_subsets' # destination directory
for zip_file in zip_files:
    base_name = os.path.basename(zip_file)[:-4]
    unzip_dir = os.path.join(extract_dir, base_name)
    os.makedirs(unzip_dir, exist_ok=True)
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)

In [7]:
from google.colab import auth   # authenticate user
auth.authenticate_user()

In [8]:
!curl https://sdk.cloud.google.com | bash

Streaming output truncated to the last 5000 lines.
google-cloud-sdk/lib/surface/sql/instances/__init__.py
google-cloud-sdk/lib/surface/sql/instances/clone.py
google-cloud-sdk/lib/surface/sql/instances/config/__init__.py
google-cloud-sdk/lib/surface/sql/instances/config/export.yaml
google-cloud-sdk/lib/surface/sql/instances/create.py
google-cloud-sdk/lib/surface/sql/instances/delete.py
google-cloud-sdk/lib/surface/sql/instances/describe.py
google-cloud-sdk/lib/surface/sql/instances/export.py
google-cloud-sdk/lib/surface/sql/instances/failover.py
google-cloud-sdk/lib/surface/sql/instances/get_latest_recovery_time.py
google-cloud-sdk/lib/surface/sql/instances/get_storage_shrink_config.py
google-cloud-sdk/lib/surface/sql/instances/import.py
google-cloud-sdk/lib/surface/sql/instances/list.py
google-cloud-sdk/lib/surface/sql/instances/patch.py
google-cloud-sdk/lib/surface/sql/instances/perform_storage_shrink.py
google-cloud-sdk/lib/surface/sql/instances/promote_replica.py
google-cloud-sdk/li

In [10]:
!gcloud init  # initialize gcloud

Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
core:
  account: madmanmarble@gmail.com
  project: big-passage-402400

Pick configuration to use:
 [1] Re-initialize this configuration [default] with new settings 
 [2] Create a new configuration
Please enter your numeric choice:  1

Your current configuration has been set to: [default]

You can skip diagnostics next time by using the following flag:
  gcloud init --skip-diagnostics

Network diagnostic detects and fixes local network connection issues.
Reachability Check passed.
Network diagnostic passed (1/1 checks passed).

Choose the account you would like to use to perform operations for this 
configuration:
 [1] madmanmarble@gmail.com
 [2] Log in with a new account
Please enter your numeric choice:  1

You are logged in as: [madmanmarble@gmail.com].

Pick cloud project to use: 
 [1] big-passage-402400
 

In [ ]:
!gsutil cp /path/to/your/local/file.jpg gs://your-bucket-name/your-destination-path/
!gsutil -m cp -r /path/to/your/local/folder gs://your-bucket-name/your-destination-path/


In [ ]:
!gsutil -m cp -r /content/image_subsets/Lanai11_subset gs://datahub_bucket/image_subsets/  # copy a file to a bucket

Copying file:///content/image_subsets/Lanai11_subset/Images/7169_8193.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Labels/6657_4609.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Images/7681_8705.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Images/1025_513.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Labels/7681_8705.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Labels/1025_513.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Images/6657_4609.tif [Content-Type=image/tiff]...
Copying file:///content/image_subsets/Lanai11_subset/Labels/7169_8193.tif [Content-Type=image/tiff]...


In [ ]:
# !gsutil cp local_file.txt gs://my-bucket/data/  # copy a file to a bucket

In [ ]:
!gsutil -m cp -r /content/image_subsets gs://datahub_bucket/image_subsets/  # copy a file to a bucket

In [ ]:
!gsutil -m mv gs://datahub_bucket/image_subsets/image_subsets/* gs://datahub_bucket/subset_data  # move files to a folder in the main bucket

Now I need to copy the csv files from my google drive, and move them here?

In [17]:
csv_files = glob.glob('/content/drive/MyDrive/wetland_unet/UNET_Image_Chips/imagechip_trainingdata/*.csv')
extract_dir = '/content/image_subsets' # destination directory
csv_list = []
for csv_file in csv_files:
    base_name = os.path.basename(csv_file)[:-4]
    csv_dir = os.path.join(extract_dir, base_name)
    base_csv = pd.read_csv(csv_file)
    base_csv['subset'] = base_name
    csv_list.append(base_csv)
    index_csv = pd.concat(csv_list, ignore_index=True)

In [18]:
csv_list

[     x_end  y_end  x_org  y_org    id           xmin           xmax  \
 0     9217   1025   8705    513    61  438833.983073  440062.758637   
 1     9729   1025   9217    513    62  440062.834396  441291.605845   
 2    13825   1025  13313    513    70  449893.209832  451122.010258   
 3    13825   1025  13313    513    70  449893.209832  451122.010258   
 4    13825   1025  13313    513    70  449893.209832  451122.010258   
 ..     ...    ...    ...    ...   ...            ...            ...   
 660  15361  16385  14849  15873  1363  453579.604836  454808.432098   
 661  15873  16385  15361  15873  1364  454808.362869  456037.183131   
 662  12289  16897  11777  16385  1400  446206.784166  447435.591764   
 663  12801  16897  12289  16385  1401  447435.568755  448664.370086   
 664  12801  16897  12289  16385  1401  447435.568755  448664.370086   
 
              ymin          ymax  label   percent         tif_name  \
 0    2.457574e+06  2.458802e+06     13  0.000919     8705_513.t

In [19]:
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).size().reset_index(name='Count')
unique_rows_df = index_csv.groupby(['tif_name', 'label', 'subset']).agg({'percent': 'mean'}).reset_index()

In [20]:
pivot_df = unique_rows_df.pivot(index=['tif_name', 'subset'], columns='label', values='percent').reset_index().fillna(0)

In [21]:
merged_df = index_csv.merge(pivot_df, on=['tif_name', 'subset'], how='left')
merged_df.drop(columns=['label'], inplace=True)

In [22]:
sum_df = merged_df.drop_duplicates(subset=['tif_name', 'subset'])

In [23]:
total_rows = len(sum_df)
train_fraction = 0.9 # modify this to set the training percentage
train_rows = int(total_rows * train_fraction)
val_rows = total_rows - train_rows

random_assignment = np.array([0] * train_rows + [1] * val_rows)
np.random.seed(42)
np.random.shuffle(random_assignment)
sum_df['random_split'] = random_assignment

<ipython-input-23-66bd3969d95d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['random_split'] = random_assignment


In [24]:
agg_df = sum_df.groupby(['random_split']).agg({13: 'mean',14:'mean',15: 'mean',16:'mean',17: 'mean',18:'mean'}).reset_index()
print(agg_df)

   random_split        13        14        15        16        17        18
0             0  0.152032  0.034274  0.011877  0.001600  0.001068  0.001292
1             1  0.145450  0.036990  0.009983  0.002517  0.001619  0.001482


In [25]:
sum_df['Images_path']="/content/image_subsets/"+sum_df['subset']+'/Images/'+sum_df['tif_name']
sum_df['Labels_path']="/content/image_subsets/"+sum_df['subset']+'/Labels/'+sum_df['tif_name']

<ipython-input-25-734faffeae8c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path']="/content/image_subsets/"+sum_df['subset']+'/Images/'+sum_df['tif_name']
<ipython-input-25-734faffeae8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path']="/content/image_subsets/"+sum_df['subset']+'/Labels/'+sum_df['tif_name']


In [26]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,...,subset,13,14,15,16,17,18,random_split,Images_path,Labels_path
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,0.000919,...,Kauai10_subset,0.000919,0.000000,0.000000,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/8...,/content/image_subsets/Kauai10_subset/Labels/8...
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,0.033161,...,Kauai10_subset,0.033161,0.000000,0.000000,0.000000,0.000000,0.000000,1,/content/image_subsets/Kauai10_subset/Images/9...,/content/image_subsets/Kauai10_subset/Labels/9...
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,0.000130,...,Kauai10_subset,0.000130,0.000137,0.003502,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/1...,/content/image_subsets/Kauai10_subset/Labels/1...
5,15361,1025,14849,513,73,453579.566028,454808.362434,2.457574e+06,2.458802e+06,0.001663,...,Kauai10_subset,0.000000,0.000000,0.001663,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/1...,/content/image_subsets/Kauai10_subset/Labels/1...
6,9217,1537,8705,1025,104,438833.962924,440062.791964,2.456345e+06,2.457574e+06,0.007381,...,Kauai10_subset,0.007381,0.000000,0.000000,0.000000,0.000000,0.000000,0,/content/image_subsets/Kauai10_subset/Images/8...,/content/image_subsets/Kauai10_subset/Labels/8...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,17921,15361,17409,14849,1079,308475.532176,309704.324054,2.156737e+06,2.157966e+06,0.012070,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.012070,0.000416,0.001926,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
6236,3585,15873,3073,15361,1087,274069.129264,275297.911026,2.155509e+06,2.156737e+06,0.000507,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000507,0.000000,0.000000,0.000000,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
6237,2049,16897,1537,16385,1156,270382.720297,271611.486128,2.153051e+06,2.154280e+06,0.000847,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000847,0.000000,0.000000,0.000000,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...
6238,17409,16897,16897,16385,1186,307246.720617,308475.512807,2.153051e+06,2.154280e+06,0.000889,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.000000,0.000000,0.000889,0,/content/image_subsets/Hawaii_2010_008_subset/...,/content/image_subsets/Hawaii_2010_008_subset/...


In [27]:
sum_df['Images_path'] = "gs://datahub_bucket/subset_data/" + sum_df['subset'] + '/Images/' + sum_df['tif_name']
sum_df['Labels_path'] = "gs://datahub_bucket/subset_data/" + sum_df['subset'] + '/Labels/' + sum_df['tif_name']

<ipython-input-27-c01b1d8b3053>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Images_path'] = "gs://datahub_bucket/subset_data/" + sum_df['subset'] + '/Images/' + sum_df['tif_name']
<ipython-input-27-c01b1d8b3053>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sum_df['Labels_path'] = "gs://datahub_bucket/subset_data/" + sum_df['subset'] + '/Labels/' + sum_df['tif_name']


In [28]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,...,subset,13,14,15,16,17,18,random_split,Images_path,Labels_path
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,0.000919,...,Kauai10_subset,0.000919,0.000000,0.000000,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,0.033161,...,Kauai10_subset,0.033161,0.000000,0.000000,0.000000,0.000000,0.000000,1,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,0.000130,...,Kauai10_subset,0.000130,0.000137,0.003502,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
5,15361,1025,14849,513,73,453579.566028,454808.362434,2.457574e+06,2.458802e+06,0.001663,...,Kauai10_subset,0.000000,0.000000,0.001663,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
6,9217,1537,8705,1025,104,438833.962924,440062.791964,2.456345e+06,2.457574e+06,0.007381,...,Kauai10_subset,0.007381,0.000000,0.000000,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,17921,15361,17409,14849,1079,308475.532176,309704.324054,2.156737e+06,2.157966e+06,0.012070,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.012070,0.000416,0.001926,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
6236,3585,15873,3073,15361,1087,274069.129264,275297.911026,2.155509e+06,2.156737e+06,0.000507,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000507,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
6237,2049,16897,1537,16385,1156,270382.720297,271611.486128,2.153051e+06,2.154280e+06,0.000847,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000847,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
6238,17409,16897,16897,16385,1186,307246.720617,308475.512807,2.153051e+06,2.154280e+06,0.000889,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.000000,0.000000,0.000889,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...


In [50]:
sum_df

,x_end,y_end,x_org,y_org,id,xmin,xmax,ymin,ymax,percent,...,subset,13,14,15,16,17,18,random_split,Images_path,Labels_path
0,9217,1025,8705,513,61,438833.983073,440062.758637,2.457574e+06,2.458802e+06,0.000919,...,Kauai10_subset,0.000919,0.000000,0.000000,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
1,9729,1025,9217,513,62,440062.834396,441291.605845,2.457574e+06,2.458802e+06,0.033161,...,Kauai10_subset,0.033161,0.000000,0.000000,0.000000,0.000000,0.000000,1,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
2,13825,1025,13313,513,70,449893.209832,451122.010258,2.457574e+06,2.458802e+06,0.000130,...,Kauai10_subset,0.000130,0.000137,0.003502,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
5,15361,1025,14849,513,73,453579.566028,454808.362434,2.457574e+06,2.458802e+06,0.001663,...,Kauai10_subset,0.000000,0.000000,0.001663,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
6,9217,1537,8705,1025,104,438833.962924,440062.791964,2.456345e+06,2.457574e+06,0.007381,...,Kauai10_subset,0.007381,0.000000,0.000000,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Kauai10_subset...,gs://datahub_bucket/subset_data/Kauai10_subset...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6233,17921,15361,17409,14849,1079,308475.532176,309704.324054,2.156737e+06,2.157966e+06,0.012070,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.012070,0.000416,0.001926,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
6236,3585,15873,3073,15361,1087,274069.129264,275297.911026,2.155509e+06,2.156737e+06,0.000507,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000507,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
6237,2049,16897,1537,16385,1156,270382.720297,271611.486128,2.153051e+06,2.154280e+06,0.000847,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000847,0.000000,0.000000,0.000000,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
6238,17409,16897,16897,16385,1186,307246.720617,308475.512807,2.153051e+06,2.154280e+06,0.000889,...,Hawaii_2010_008_subset,0.000000,0.000000,0.000000,0.000000,0.000000,0.000889,0,gs://datahub_bucket/subset_data/Hawaii_2010_00...,gs://datahub_bucket/subset_data/Hawaii_2010_00...
